In [6]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 618.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 605.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 591.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 490.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 507.2 kB/s eta 0:00:00a 0:00:01


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# The target URL
url = 'https://qmrawiki.org/node/402'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find elements by HTML tags, attributes, or CSS class
    titles = soup.find_all('h1')
    for title in titles:
        print(title.get_text())
    tables_data = soup.find_all('table')
    
    for table_data in tables_data[3]:
        table_row_data = [row for row in table_data.get_text().split('\n') if row != '']
        if len(table_row_data):
            columns = table_row_data[1:5]
            experiment_data = []
            print(columns)
            i = 5
            while i <= len(table_row_data):
                experiment_data
                row_data = table_row_data[i:i+4]
                i+=4 
                if(len(row_data) > 0):
                    experiment_data.append(row_data)
    df = pd.DataFrame(columns=columns, data=experiment_data)
    print(df)         
else:
    print('Failed to retrieve the webpage')



Optimization Output for Exp. 183
['Dose', 'Infected', 'Non-infected', 'Total']
   Dose Infected Non-infected Total
0   100        2            2     4
1   300        2            3     5
2  1000        1            2     3
3  3000        3            1     4


In [12]:
url_new = 'https://qmrawiki.org/experiments/cryptosporidium-parvum'

response = requests.get(url_new)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    experiments_container = soup.find(class_='view-pathogen-dose-response-experiments')
    
    #experiments = experiments_container.find_all(class_='response-table-inner')

    #for experiment in experiments:
        #print(experiment.get_text().split(' '))
        
    

['\n\n108', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'oral', '', '', '', '', '', '', '', '', '', '', '', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', '8', '', '', '', '', '', '', '', '', '', '', '', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Iowa', 'strain', '', '', '', '', '', '', '', '', '', '', '', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'oocysts', '', '', '', '', '', '', '', '', '', '', '', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'human', '', '', '', '', '', '', '', '', '', '', '', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'exponential', '', '', '', '', '', '', '', '', '', '', '', '\n\n\nk', '=', '4.19E-03\nLD50/ID50', '=', '1.65E+02', '\n', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'infection', '', '', '', '', '', '', '', '', '', '', '', '\n\nDuPont,', 'H', 'L.,', 'et', 'al.', '"The', 'infectivity', 'of', 'Cryptosporidium', 'parvum', 'in', 'healthy', 'vol

# Step 1, iterating over all the dose response pages and going to next page

In [9]:
dose_responses_url = 'https://qmrawiki.org/framework/dose-response/experiments'
response = requests.get(dose_responses_url)

In [10]:
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    

In [17]:
individual_dose_responses_list = soup.find_all(class_='quicktabs-views-group')
for list in individual_dose_responses_list:
    dose_response_url = list.find_all('span')
    base_url = 'https://qmrawiki.org'
    for dose_responses in tqdm(dose_response_url):
        temps = dose_responses.find_all('a')
        for temp in temps:
            experiment_overview_url = temp.get('href')
            print(base_url + experiment_overview_url)
            if base_url + experiment_overview_url not in ['https://qmrawiki.org/experiments/influenza/104'
                                                          , 'https://qmrawiki.org/experiments/variola-major',
                                                          'https://qmrawiki.org/experiments/listeria-monocytogenes/113',
                                                          'https://qmrawiki.org/experiments/pseudomonas-aeruginosa/460',
                                                          'https://qmrawiki.org/experiments/lassa-virus/%2B%2B']:
                extract_from_dose_response_page(base_url+ experiment_overview_url, base_url)
                
        

  0%|          | 0/1 [00:00<?, ?it/s]

https://qmrawiki.org/experiments/acanthamoeba/103
Dose response has inline dataset
Acanth_Cornea1
Dose response has inline dataset
Acanth_Cornea2
https://qmrawiki.org/experiments/acanthamoeba/109
Data unavailable for Acanth_Intranasal_Pooled
Acanth_Intranasal_Pooled
Dose response has inline dataset
Acanth_Intranasal2
Dose response has inline dataset
Acanth_Intranasal3
Dose response has inline dataset
Acanth_Intranasal4
Dose response has inline dataset
Acanth_Intranasal1
https://qmrawiki.org/experiments/cryptosporidium-hominis
[['Dose', 'Infected', 'Non-infected', 'Total'], ['30', '1', '4', '5'], ['100', '3', '5', '8'], ['300', '2', '1', '3'], ['500', '5', '1', '6'], ['1000', '2', '0', '2'], ['1E+04', '3', '0', '3'], ['1E+05', '1', '0', '1'], ['1E+06', '1', '0', '1']]
108
[['Dose', 'Infected', 'Not infected', 'Total'], ['30', '2', '3', '5'], ['100', '4', '4', '8'], ['300', '2', '1', '3'], ['500', '5', '1', '6'], ['1000', '2', '0', '2'], ['1E+04', '3', '0', '3'], ['1E+05', '1', '0', '1']

100%|██████████| 1/1 [00:08<00:00,  8.98s/it]


[['Dose', 'Dead', 'Survived', 'Total'], ['1E+02', '0', '10', '10'], ['1E+02', '0', '10', '10'], ['1E+02', '0', '10', '10'], ['1E+03', '0', '10', '10'], ['1E+03', '1', '9', '10'], ['1E+03', '1', '9', '10'], ['1E+04', '1', '9', '10'], ['1E+04', '4', '6', '10'], ['1E+04', '4', '6', '10'], ['1E+05', '4', '6', '10'], ['1E+05', '6', '4', '10'], ['1E+05', '7', '3', '10'], ['1E+06', '7', '3', '10']]
954,955,956,957


  0%|          | 0/1 [00:00<?, ?it/s]

...

[['Dose', 'Dead', 'Survived', 'Total'], ['2', '0', '20', '20'], ['8', '5', '15', '20'], ['26', '13', '7', '20'], ['74', '9', '1', '10'], ['257', '10', '0', '10']]
3


  0%|          | 0/1 [00:00<?, ?it/s]

https://qmrawiki.org/experiments/prion
[['Dose', 'Dead', 'Survived', 'Total'], ['200', '0', '40', '40'], ['2000', '1', '79', '80'], ['2E+04', '9', '71', '80'], ['2E+05', '58', '22', '80'], ['2E+06', '29', '1', '30']]
250
[['Dose', 'Dead', 'Survived', 'Total'], ['125', '0', '11', '11'], ['1250', '1', '9', '10'], ['12500', '2', '8', '10']]
251


100%|██████████| 1/1 [00:00<00:00,  1.05it/s]

[['Dose', 'Infected', 'Non-infected', 'Total'], ['0.0186', '0', '13', '13'], ['0.186', '4', '12', '16'], ['1.86', '9', '5', '14'], ['18.6', '13', '0', '13']]
252


In [7]:
def optimal_params_parser(params_list):
    params = []
    values = []
    params_list = [p for p in params_list if p!='']
    for param in params_list:
        #print(param)
        _ = param.split(' ')
        params.append(_[0])
        _ = [__ for __ in _ if __ != '']
        if ',' in _[2]:
            values.append(float(_[2].replace(',', '')))
        else:
            if '"' in _[2]:
                values.append(float(_[2].replace('"', '')))
            else:
                values.append(float(_[2]))
    return [params, values]

In [13]:
def extract_metadata(soup, experiment_id):
        
        print(experiment_id)
        experiment_exposure_route = soup.select('td.views-field-field-exposure-route')
        for route in experiment_exposure_route:
            exposure_route = route.get_text().strip()
        experiment_strain_container = soup.select('td.views-field-field-agent-strain')
        for strain in experiment_strain_container:
            experiment_strain = strain.get_text().strip()
        experiment_doseunits_container = soup.select('td.views-field-field-dose-units')
        for dose in experiment_doseunits_container:
            experiment_dose_units = dose.get_text().strip()
        experiment_host_container = soup.select('td.views-field-field-host-type')
        for host in experiment_host_container:
            experiment_host = host.get_text().strip()
        experiment_modeltype_container = soup.select('td.views-field-field-best-fit-model')
        for modeltype in experiment_modeltype_container:
            experiment_model = modeltype.get_text().strip()
        #TODO: Create this into a separate function
        experiment_params_container = soup.select('td.views-field-nothing')
        for param in experiment_params_container:
            experiment_optimal_params, experiment_optim_vals = optimal_params_parser(param.get_text().strip().split('\n'))
        ########
        experiment_response_cont = soup.select('td.views-field-field-response')
        for response in experiment_response_cont:
            experiment_response_type = response.get_text().strip()

        metadata_cols = ['Experiment Id', 'Exposure Route', 'Agent Strain', 'Dose Units', 'Host', 'Best Fit Model', f'Optimized Params {" ".join(experiment_optimal_params)}', 
                            'Response Type']
        metadata_data = [experiment_id, exposure_route, experiment_strain, experiment_dose_units, experiment_host,
                        experiment_model, experiment_optim_vals, experiment_response_type]
            
        metadata_df = pd.DataFrame([metadata_data], columns=metadata_cols)
        
        metadata_df.to_csv(f"data/metadata/{experiment_id}_metadata.csv")

In [15]:
#TODO: Have to adjust for pages that have the experiment data inside Experiment Dataset
def extract_from_dose_response_page(url, base_url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        #Getting experiment table data
        experimentId_container = soup.select('td.views-field-field-experiment-id-text')
        
        for container in experimentId_container:
            experiment_url = container.find('a').get('href')
            experiment_id = container.find('a').get_text()            
            extract_from_experiment_page(base_url + experiment_url, experiment_id)
            
        
        #Getting experiment META data
        
            extract_metadata(soup, experiment_id)
        


In [3]:
def extract_datatable(data, exp_id):
    
    soup = BeautifulSoup(data, 'html.parser')

    table = soup.find('table', {'border' : 1})
    data_arr = []
    if table:
        for t in table:
        
            arr = t.get_text().strip().split('\n')
            
            
            if len(arr) > 1:
                arr = [a for a in arr if a != '']
                
                k=0
                while k <= len(arr):
                    data_arr.append(arr[k:k+4])
                    k+=4
                data_arr = [item for item in data_arr if item !=[]]
                print(data_arr)
                return data_arr

    else:
        print(f'Data unavailable for {exp_id}')
    

In [1]:
def extract_from_experiment_page(url, exp_id):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        experiment_dataset = soup.find_all(id='tablefield-0')

        for data in experiment_dataset:
            experiment_data_table = data.find_all('tr')
            if(len(experiment_data_table) > 0):
                print('Dose response has inline dataset')
                df = parse_meta_table(experiment_data_table, response.text)
                df.to_csv(f'data/{exp_id}_data.csv')
                #TODO: Add function here to extract the data if the dataset is embeded
            else:
                experiment_data_html = extract_datatable(response.text, exp_id)
                if experiment_data_html:
                    #print(experiment_data_html[1:])
                    df = pd.DataFrame(experiment_data_html[1:],columns=experiment_data_html[0])
                    df.to_csv(f'data/{exp_id}_data.csv')


In [2]:
def parse_meta_table(data, page_txt):
    i = 0
    soup = BeautifulSoup(page_txt, 'html.parser')
    odds = soup.select('tr.odd')    
    evens = soup.select('tr.even')
    columns = soup.select('th.row_0')
    column_names = [column.get_text() for column in columns]
    rows = []
    even_counter = 2
    odd_counter = 1
    for even in evens:
        
        row = []
        for n in even.find_all(class_=f'row_{even_counter}'):
    
            row.append(n.get_text())
        even_counter += 2
        rows.append(row)
    for odd in odds:
        row = []
        
        for n in odd.find_all(class_=f'row_{odd_counter}'):
            
            row.append(n.get_text())
        rows.append(row)     
        odd_counter += 2
    df = pd.DataFrame(columns=column_names, data=rows) 
    return df.sort_values(by=df.columns[0])
    
    
        

In [ ]:
#TODO: